In [2]:
from src.mazeData import *
from src.maze2025GraphClass import mazeGraph
from pyvis.network import Network
from src.PS_agentPrograms import BestFirstSearchAgentProgram
from src.mazeProblemClass import MazeProblem
from src.agents import ProblemSolvingMazeAgentBFS
from src.naigationEnvironmentClass import MazeNavigationEnvironment
from src.Lab4Environment import *

In [4]:
def makeMaze(n):
  size = (n,n)
  proba_0 =0.25 # resulting array will have 25%? of zeros
  proba_Enemy =0.1 # resulting array will have 10%? of enemy
  arrMaze=np.random.choice([0, 1, 2], size=size, p=[proba_0, 1-proba_0-proba_Enemy, proba_Enemy] )
  return arrMaze

initState = (0,0)
goalState = (6,6)

mazeSize=7
mainMaze = makeMaze(mazeSize)
MazeCheck(mainMaze,initState,goalState)
mazeAvalActs=defineMazeAvailableActions(mainMaze)
maze1TM=makeMazeTransformationModel(mazeAvalActs)
mazeWorldGraph=mazeGraph(maze1TM, mazeStatesLocations(list(maze1TM.keys())))

maze1TM

{(0, 0): {'right': (1, 0)},
 (0, 1): {'left': (0, 0), 'down': (1, 1)},
 (0, 2): {},
 (0, 3): {'right': (4, 0), 'down': (3, 1)},
 (0, 4): {'left': (3, 0), 'down': (4, 1)},
 (0, 5): {},
 (0, 6): {},
 (1, 0): {},
 (1, 1): {'up': (1, 0), 'right': (2, 1), 'down': (1, 2)},
 (1, 2): {'left': (1, 1), 'right': (3, 1), 'down': (2, 2)},
 (1, 3): {'left': (2, 1), 'up': (3, 0), 'right': (4, 1), 'down': (3, 2)},
 (1, 4): {'left': (3, 1), 'up': (4, 0), 'right': (5, 1), 'down': (4, 2)},
 (1, 5): {'left': (4, 1), 'right': (6, 1)},
 (1, 6): {'left': (5, 1), 'down': (6, 2)},
 (2, 0): {},
 (2, 1): {'up': (1, 1), 'right': (2, 2), 'down': (1, 3)},
 (2, 2): {'left': (1, 2), 'up': (2, 1), 'right': (3, 2), 'down': (2, 3)},
 (2, 3): {'left': (2, 2), 'up': (3, 1), 'right': (4, 2)},
 (2, 4): {'left': (3, 2), 'up': (4, 1)},
 (2, 5): {},
 (2, 6): {'up': (6, 1), 'down': (6, 3)},
 (3, 0): {},
 (3, 1): {'up': (1, 2), 'right': (2, 3)},
 (3, 2): {'left': (1, 3), 'up': (2, 2), 'down': (2, 4)},
 (3, 3): {},
 (3, 4): {},
 

In [ ]:
mainMaze

In [ ]:
maze1TM

In [ ]:
mazeAvalActs

In [ ]:
net_maze = Network(heading="Lab 4 Maze",
                bgcolor ="#242020",
                font_color = "white",
                height = "750px",
                width = "100%" 
)
nodeColors={
    "wall":"red",
    "path": "White",
    "Goal": "Green",
    "Start": "Yellow",
    "Enemy": "Orange"
}
nodeColorsList=[]

for node in mazeWorldGraph.origin.keys():
    if mainMaze[node[0],node[1]]==1:
        nodeColorsList.append(nodeColors["path"])
    elif mainMaze[node[0],node[1]]==0:
        nodeColorsList.append(nodeColors["wall"])
    else:
        nodeColorsList.append(nodeColors["Enemy"])

nodes=["-".join(str(item) for item in el) for el in mazeWorldGraph.origin.keys()]

x_coords = []
y_coords = []

for node in mazeWorldGraph.origin.keys():
    x,y=mazeWorldGraph.getLocation(node)
    x_coords.append(x)
    y_coords.append(y)

sizes=[10]*len(nodes)
net_maze.add_nodes(nodes, color=nodeColorsList, x=x_coords, y=y_coords, size=sizes, title=nodes)

for node in net_maze.nodes:
    node['label']=''

edge_weights = {(intTupleTostr(k), intTupleTostr(v2)) : k2 for k, v in mazeWorldGraph.origin.items() for k2, v2 in v.items()}

edges=[]

for node_source in mazeWorldGraph.nodes():
    for node_target, action in mazeWorldGraph.get(node_source).items():
        #node_target or node_source is a tuple -> convert to str
        if (intTupleTostr(node_source),intTupleTostr(node_target)) not in edges and (intTupleTostr(node_target), intTupleTostr(node_source)):
            net_maze.add_edge(intTupleTostr(node_source),intTupleTostr(node_target), label=edge_weights[(intTupleTostr(node_source),intTupleTostr(node_target))])
            edges.append((intTupleTostr(node_source),intTupleTostr(node_target)))

for node in net_maze.nodes:
    if node['id']==intTupleTostr(goalState):
        node['color']=nodeColors['Goal']
    elif node['id']==intTupleTostr(initState):
        node['color']=nodeColors['Start']


In [ ]:
net_maze.toggle_physics(False)

In [ ]:
mp1=MazeProblem(initState, goalState, mazeWorldGraph)

In [ ]:
BFSAgent=BestFirstSearchAgentProgram()

In [ ]:
BFSmazeAgent1=ProblemSolvingMazeAgentBFS(initState, mazeWorldGraph, goalState)

In [ ]:
#maze_Env1=Lab4NavEnvironment(mazeWorldGraph)

maze_Env1=MazeNavigationEnvironment(mazeWorldGraph)

In [ ]:
maze_Env1.add_thing(BFSmazeAgent1)
#if it runs X times change a wall to a path

In [ ]:
maze_Env1.run()

In [ ]:
net_maze.show("graphMaze1.html", notebook=False)